<h2 align="center">点击下列图标在线运行HanLP</h2>
<div align="center">
	<a href="https://colab.research.google.com/github/hankcs/HanLP/blob/doc-zh/plugins/hanlp_demo/hanlp_demo/zh/srl_restful.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
	<a href="https://mybinder.org/v2/gh/hankcs/HanLP/doc-zh?filepath=plugins%2Fhanlp_demo%2Fhanlp_demo%2Fzh%2Fsrl_restful.ipynb" target="_blank"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>
</div>

## 安装

无论是Windows、Linux还是macOS，HanLP的安装只需一句话搞定：

In [ ]:
!pip install hanlp_restful -U

## 创建客户端

In [1]:
from hanlp_restful import HanLPClient
HanLP = HanLPClient('https://www.hanlp.com/api', auth=None, language='zh') # auth不填则匿名，zh中文，mul多语种

#### 申请秘钥
由于服务器算力有限，匿名用户每分钟限2次调用。如果你需要更多调用次数，[建议申请免费公益API秘钥auth](https://bbs.hanlp.com/t/hanlp2-1-restful-api/53)。

## 语义角色分析
任务越少，速度越快。如指定仅执行语义角色分析：

In [2]:
doc = HanLP('2021年HanLPv2.1为生产环境带来次世代最先进的多语种NLP技术。', tasks='srl')

返回值为一个[Document](https://hanlp.hankcs.com/docs/api/common/document.html):

In [3]:
print(doc)

{
  "tok/fine": [
    ["2021年", "HanLPv2.1", "为", "生产", "环境", "带来", "次", "世代", "最", "先进", "的", "多", "语种", "NLP", "技术", "。"]
  ],
  "srl": [
    [[["2021年", "ARGM-TMP", 0, 1], ["HanLPv2.1", "ARG0", 1, 2], ["为生产环境", "ARG2", 2, 5], ["带来", "PRED", 5, 6], ["次世代最先进的多语种NLP技术", "ARG1", 6, 15]], [["次世代", "ARGM-TMP", 6, 8], ["最", "ARGM-ADV", 8, 9], ["先进", "PRED", 9, 10], ["NLP技术", "ARG0", 13, 15]]]
  ]
}


`doc['srl']`字段为语义角色标注结果，每个四元组的格式为`[论元或谓词, 语义角色标签, 起始下标, 终止下标]`。其中，谓词的语义角色标签为`PRED`，起止下标对应以`tok`开头的第一个单词数组。

可视化谓词论元结构：

In [4]:
doc.pretty_print()

Token    	SRL PA1     	Token    	SRL PA2     
─────────	────────────	─────────	────────────
2021年    	───►ARGM-TMP	2021年    	            
HanLPv2.1	───►ARG0    	HanLPv2.1	            
为        	◄─┐         	为        	            
生产       	  ├►ARG2    	生产       	            
环境       	◄─┘         	环境       	            
带来       	╟──►PRED    	带来       	            
次        	◄─┐         	次        	◄─┐         
世代       	  │         	世代       	◄─┴►ARGM-TMP
最        	  │         	最        	───►ARGM-ADV
先进       	  │         	先进       	╟──►PRED    
的        	  ├►ARG1    	的        	            
多        	  │         	多        	            
语种       	  │         	语种       	            
NLP      	  │         	NLP      	◄─┐         
技术       	◄─┘         	技术       	◄─┴►ARG0    
。        	            	。        	            


遍历谓词论元结构：

In [9]:
for i, pas in enumerate(doc['srl'][0]):
    print(f'第{i+1}个谓词论元结构：')
    for form, role, begin, end in pas:
        print(f'{form} = {role} at [{begin}, {end}]')

第1个谓词论元结构：
2021年 = ARGM-TMP at [0, 1]
HanLPv2.1 = ARG0 at [1, 2]
为生产环境 = ARG2 at [2, 5]
带来 = PRED at [5, 6]
次世代最先进的多语种NLP技术 = ARG1 at [6, 15]
第2个谓词论元结构：
次世代 = ARGM-TMP at [6, 8]
最 = ARGM-ADV at [8, 9]
先进 = PRED at [9, 10]
NLP技术 = ARG0 at [13, 15]


为已分词的句子执行语义角色分析：

In [11]:
HanLP(tokens=[
    ["HanLP", "为", "生产", "环境", "带来", "次世代", "最", "先进", "的", "多语种", "NLP", "技术", "。"],
    ["我", "的", "希望", "是", "希望", "张晚霞", "的", "背影", "被", "晚霞", "映红", "。"]
  ], tasks='srl', skip_tasks='tok*').pretty_print()

Token	SRL PA1 	Token	SRL PA2     
─────	────────	─────	────────────
HanLP	───►ARG0	HanLP	            
为    	◄─┐     	为    	            
生产   	  ├►ARG2	生产   	            
环境   	◄─┘     	环境   	            
带来   	╟──►PRED	带来   	            
次世代  	◄─┐     	次世代  	───►ARGM-TMP
最    	  │     	最    	───►ARGM-ADV
先进   	  │     	先进   	╟──►PRED    
的    	  ├►ARG1	的    	            
多语种  	  │     	多语种  	            
NLP  	  │     	NLP  	            
技术   	◄─┘     	技术   	───►ARG0    
。    	        	。    	            

Tok	SRL PA1 	Tok	SRL PA2 	Tok	SRL PA3 
───	────────	───	────────	───	────────
我  	◄─┐     	我  	        	我  	        
的  	  ├►ARG0	的  	        	的  	        
希望 	◄─┘     	希望 	        	希望 	        
是  	╟──►PRED	是  	        	是  	        
希望 	◄─┐     	希望 	╟──►PRED	希望 	        
张晚霞	  │     	张晚霞	◄─┐     	张晚霞	◄─┐     
的  	  │     	的  	  │     	的  	  ├►ARG1
背影 	  ├►ARG1	背影 	  │     	背影 	◄─┘     
被  	  │     	被  	  ├►ARG1	被  	        
晚霞 	  │     	晚霞 	  │     	晚霞 	───►ARG0
映红 	◄─┘     	映红 	◄─┘ 